## Food LLM annotation

In [1]:
import pandas as pd
import os
import base64
import requests
import re
import numpy as np
import ollama

In [5]:
#picture 008 in patient 004, 008, 009, 010, 016 removed. Double
#picture 021 in patient 001 removed. Double

In [24]:
macronutrients_instruction = '''Examine the provided meal image to analyze and estimate its nutritional content accurately. Focus on determining the amounts of simple sugars (like industrial sugar and honey), 
complex sugars (such as starch and whole grains), proteins, fats, and dietary fibers (found in fruits and vegetables), all in grams. Also estimate the total weight of the meal in grams.
To assist in accurately gauging the scale of the meal, a 1 Swiss Franc coin, which has a diameter of 23.22 mm, may be present in the picture. 
Use the size of this coin as a reference to estimate the size of the meal and the amounts of the nutrients more precisely. 
Provide your assessment of each nutritional component in grams. All estimates should be given as a single whole number. If there is no coin in the picture or the meal is covered partially, estimate anyways.
Format your response as follows:
- Simple sugars (g): 
- Complex sugars (g): 
- Proteins (g): 
- Fats (g): 
- Dietary fibers (g): 
- Weight (g): 
- Explanation: 

Example response:
Simple sugars (g): 40
Complex sugars (g): 60
Proteins (g): 25
Fats (g): 30
Dietary fibers (g): 5 
Weight (g): 750
Explanation: The pizza and cola meal, with its refined crust and toppings, is rich in carbs, fats, and proteins. The cola boosts the meal's simple sugars. 
The 1 Swiss Franc coin helps estimate the pizza at 30 cm diameter and the cola at 330 ml, indicating a significant blood sugar impact.'''

In [26]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')
  
def parse_nutritional_info(text):
    pattern = r'(Simple sugars \(g\)|Complex sugars \(g\)|Proteins \(g\)|Fats \(g\)|Dietary fibers \(g\)|Weight \(g\)):\s*(\d+)'
    matches = re.findall(pattern, text)
    nutritional_info = {match[0]: int(match[1]) for match in matches}
    simple_sugars = nutritional_info.get('Simple sugars (g)', 0)
    complex_sugars = nutritional_info.get('Complex sugars (g)', 0)
    proteins = nutritional_info.get('Proteins (g)', 0)
    fats = nutritional_info.get('Fats (g)', 0)
    dietary_fibers = nutritional_info.get('Dietary fibers (g)', 0)
    weight = nutritional_info.get('Weight (g)', 0)
    return simple_sugars, complex_sugars, proteins, fats, dietary_fibers, weight

In [38]:
for patient in ['001', '002', '004', '006', '007', '008']:
    print(f"Processing patient {patient}")
    food_data = pd.read_csv(f'diabetes_subset_pictures-glucose-food-insulin/{patient}/food.csv')
    food_data = food_data[['picture', 'datetime']]
    food_data[['simple_sugars', 'complex_sugars', 'proteins', 'fats', 'dietary_fibers', 'weight', 'message']] = 0
    for i, row in food_data.iterrows():
        image_path = f"diabetes_subset_pictures-glucose-food-insulin/{patient}/food_pictures/{row['picture']}"
        if not os.path.exists(image_path):
            print(f"Image {row['picture']} missing") 
            next
        base64_image = encode_image(image_path)
        res = ollama.chat(
            model="llava:v1.6",
            messages=[
                {
                    'role': 'user',
                    'content': macronutrients_instruction,
                    'images': [image_path]
                }
            ]
        )
        message = res['message']['content']
        try:
            parsed_info = parse_nutritional_info(message)
            print(parsed_info)
            food_data.loc[i, ['simple_sugars', 'complex_sugars', 'proteins', 'fats', 'dietary_fibers', 'weight']] = parsed_info
            food_data.loc[i, 'message'] = message
        except:
            print(f"Picture {row['picture']} for patient {patient} could not be annotated")
    food_data.to_csv(f'llava16_food_data/food_data_{patient}.csv', index=False)

Processing patient 008
(20, 15, 10, 30, 5, 750)
(20, 30, 5, 10, 2, 750)
(50, 100, 20, 30, 10, 540)
(60, 120, 15, 20, 3, 200)
(50, 60, 20, 30, 5, 1700)
(50, 75, 15, 25, 5, 625)
(50, 120, 35, 15, 8, 700)
(50, 70, 20, 10, 5, 650)
(40, 60, 25, 30, 5, 750)
(50, 100, 40, 20, 5, 750)
(12, 48, 8, 3, 5, 0)
(50, 100, 20, 30, 5, 730)
(25, 50, 15, 10, 3, 98)
(30, 50, 15, 20, 5, 90)
(60, 30, 15, 20, 5, 850)
